## Graph Neural Networks with Pytorch
## Target: Graph Diffusion Convolution
- Original Paper: https://arxiv.org/abs/1911.05485
- Original Code: https://github.com/rusty1s/pytorch_geometric/blob/master/examples/gcn.py

In [1]:
import os
import sys

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv

sys.path.append('../')
from utils import *
logger = make_logger(name='gdc_logger')

In [2]:
# Load Dataset
dataset = 'Cora'
path = os.path.join(os.getcwd(), '..', 'data', dataset)
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]

logger.info(f"원래 데이터 Edge Index 형상: {data.edge_index.shape}")

2021-08-11 22:14:27,648 - gdc_logger - 원래 데이터 Edge Index 형상: torch.Size([2, 10556])


**GDC: Graph Diffusion Convolution**은 `torch_geometric.transforms`에 있는 `GDC` class를 통해 간단히 적용할 수 있다.  
**GDC**는 오직 direct 이웃 만을 사용하는 기존의 GNN의 제한점을 제거하고 powerful하면서도 spatially localized한 Graph 합성곱이다.  

1-hop Neighbor 만으로 특정 node와 그 node의 이웃과의 관계를 정의하는 것은, Noise가 많고 복잡한 Graph 구조에서는 한계를 드러내는 경우가 있다.  
**GDC**는 더 큰 이웃 집합에서 정보를 통합함으로써 이를 극복한다.  

**GDC**는 아래와 같이 크게 4가지의 과정을 거쳐 수행된다.  

1) Transition Matrix $\mathbf{T}$ 를 계산한다.  
2) 아래 식을 통해 $\mathbf{S}$를 얻는다.  

$$ \mathbf{S} = \Sigma_{k=0}^{\infty} \theta_{k} \mathbf{T}^k $$  

3) 특정 임계 값을 넘지 못하는 작은 값들은 0으로 만듦으로써 위 결과물을 sparsify한다.  
(Top-K 방식을 채택할 수도 있다.)  

$$ \tilde{\mathbf{S}} = spar(\mathbf{S}) $$  

4) 최종적으로 Transition Matrix $\mathbf{T}_{\tilde{\mathbf{S}}}$ 를 계산한다.  

$$ \mathbf{T}_{\tilde{\mathbf{S}}} = \mathbf{D}^{-\frac{1}{2}}_{\tilde{\mathbf{S}}} \tilde{\mathbf{S}} \mathbf{D}^{-\frac{1}{2}}_{\tilde{\mathbf{S}}} $$  


사실 위 과정을 쉽게 풀어서 설명하는 것은 쉽지는 않다.  
따라서 반드시 [논문 원본](https://arxiv.org/abs/1911.05485)을 확인하는 것을 권하며 한글로 작성된 리뷰 글은 [이 곳](https://greeksharifa.github.io/machine_learning/2021/08/12/GDC/)에서 확인할 수 있다.  

간단하게만 짚고 넘어가자면, 초기의 Transition Matrix는 대칭 버전 기준으로 $\mathbf{D}^{-\frac{1}{2}} \mathbf{A} \mathbf{D}^{-\frac{1}{2}}$ 로 계산되는데,  
여기까지는 기본적인 Vanilla Graph Convolution과 다를 바가 없다.  

그런데 여기서 **Diffusion**, 즉 확산을 적용하기 위해서 가중치 계수 $\theta_k$ 가 등장한다.  
참고로 이 가중치 계수에는 여러 버전이 적용될 수 있고, 논문에서는 **PPR**과 **Heat Kernel** 버전을 자세히 소개하고 있다.  

본 코드에서는 **PPR** 버전을 사용한다.    

반복 계산을 통해 일정 수준으로 수렴하게 되면, $\mathbf{S}$ 는 $\mathbf{T}$ 와 달리 direct 1-hop 이웃 관계만을 표현한 것이 아니라,
보다 넓은 범위로 확산된 이웃 관계를 표현한 Transition Matrix로 탈바꿈하게 된다.  

이 때 이 $\mathbf{S}$ 를 바로 사용하게 되면 $\mathbf{T}$ 처럼 sparse한 형태가 아니라 대부분의 값이 0이 아닌 dense한 행렬이 된다.  
따라서 임계값을 넘지 못하는 작은 값의 경우, 약한 관계라고 규정하고 이를 0으로 masking하는 작업을 거치게 된다.  (Sparsification)  

Step4에서 다시 한번 Transition Matrix를 계산하고 이를 통해 Graph 합성곱을 진행하는 것이 **GDC**의 수행 과정이다.  

간단한 설명이 끝났으니 사용법을 확인해본다.  

In [ ]:
# setting GDC
gdc = T.GDC(self_loop_weight=1, normalization_in='sym',
            normalization_out='col',
            diffusion_kwargs=dict(method='ppr', alpha=0.05),
            sparsification_kwargs=dict(method='topk', k=128, dim=0), exact=True)

**GDC**의 인자는 논문을 읽어보지 않으면 이해하기가 조금 어렵다.  

`self_loop_weight`는 기본값이 1이며, self-loop edge를 더할 때의 가중치를 의미한다.  

`normalization_in` 인자의 기본값은 `sym`이며 초기의 Transition Matrix를 어떻게 normalize할지에 대한 설정값이다. 다른 가능한 값으로는 `col`과 `row`가 있다.  

`sym`의 경우 앞서 설명했던 방식으로 계산된다.  

`row`는 아래와 같이 계산된다.  

$$ \mathbf{T} = \mathbf{A} \mathbf{D}^{-\frac{1}{2}} $$  

`col`은 아래와 같이 계산된다.  

$$ \mathbf{T} = \mathbf{D}^{-\frac{1}{2}} \mathbf{A} $$  

`normalization_out` 인자는 위와 같은 설정이나 최종 Transition Matrix에 적용된다.  

`diffusion_kwargs`는 앞서 설명했던 가중치 계수 $\theta$ 를 어떻게 만들 것인가에 대한 설정 값을 딕셔너리 형태로 입력한다. **ppr** 외에도 **heat** 혹은 **coeff**를 입력할 수 있다.  

참고로 **ppr**의 Hyper-parameter인 $\alpha$ 의 기본 값은 0.15이다.  

`sparsification_kwargs`는 희소화하는 방식과 세부 설정 값을 딕셔너리로 입력하는 인자이다. `threshold`와 `topk`를 적용할 수 있다.  

`diffusion_matrix_approx`, `diffusion_matrix_exact`, `sparsify_dense`, `sparsify_sparse`, `transition_matrix`와 같은 메서드 또한 사용할 수 있으니 [공식 문서](https://pytorch-geometric.readthedocs.io/en/latest/modules/transforms.html#torch_geometric.transforms.GDC)를 참조하길 바란다.  

논문의 저자는 Github에서 **torch_geometric**을 사용할 것을 권하고 있긴 하지만,  
[이 곳](https://github.com/klicperajo/gdc/blob/master/gdc_demo.ipynb)에서 **GDC**에 대한 코드를 공개하고 있으니, 확인하면 좋을 것이다.  

데이터에 **GDC**을 적용하는 것은 아래와 같이 매우 간단하다.

In [11]:
data = gdc(data)

logger.info(f"GDC 적용 후 Edge Index 형상: {data.edge_index.shape}")

2021-08-11 23:06:13,893 - gdc_logger - GDC 적용 후 Edge Index 형상: torch.Size([2, 346624])


In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(
            dataset.num_features, 16, cached=True, normalize=False)
        self.conv2 = GCNConv(
            16, dataset.num_classes, cached=True, normalize=False)

    def forward(self):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return F.log_softmax(x, dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=5e-4),
    dict(params=model.conv2.parameters(), weight_decay=0)
], lr=0.01)

위와 같이 GNN 모델을 선언하고 나면, 이후 학습 과정은 다른 예시와 동일하다.  